In [1]:
# Cell 1: Imports and Global Variables
import pandas as pd
import os
import numpy as np
import cv2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from functools import lru_cache
import matplotlib.pyplot as plt
%matplotlib inline

# Global variables
IMAGE_HEIGHT = 32
IMAGE_WIDTH = 32
IMAGE_CHANNELS = 3
BATCH_SIZE = 10
TIMESTEPS = 24
ECHPOS_SIZE=150

In [2]:
@lru_cache(maxsize=100000)
def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not load image at {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    # Normalize pixel values to [0, 1] instead of VGG16 preprocessing
    img_normalized = img_resized / 255.0
    return img_normalized

def load_images_with_data(image_dir="F:/Pheonix/Thesis/Project/Code/AQI Index/.venv/Dev/Data/cleaned_folder",
                         csv_file="F:/Pheonix/Thesis/Project/Code/AQI Index/.venv/Dev/Data/cleaned_data.csv"):
    df = pd.read_csv(csv_file)
    image_data_list = []
    
    for _, row in df.iterrows():
        image_name = row['Filename']
        image_path = os.path.join(image_dir, image_name)
        if os.path.exists(image_path):
            try:
                img_resized = load_and_preprocess_image(image_path)
                values = row.to_dict()
                values['image'] = img_resized
                image_data_list.append(values)
            except Exception as e:
                print(f"Error loading image {image_name}: {e}")
    return image_data_list

image_data_list = load_images_with_data()
print(f"Loaded {len(image_data_list)} images")

Loaded 10610 images


In [3]:
def prepare_dataset(image_data_list):
    """Prepare dataset with whole images, no segmentation."""
    X = []
    y_dict = {'AQI': [], 'PM2.5': [], 'PM10': [], 'O3': [], 'CO': [], 'SO2': [], 'NO2': []}
    sequence_X = []
    scalers = {}
    
    for i, data in enumerate(image_data_list):
        img = data['image']
        if img.shape != (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS):
            img = cv2.resize(img, (IMAGE_WIDTH, IMAGE_HEIGHT))
        
        sequence_X.append(img)
        for param in y_dict.keys():
            y_dict[param].append(data[param])
        
        if (i + 1) % TIMESTEPS == 0:
            X.append(np.array(sequence_X))
            sequence_X = []
    
    if sequence_X:
        remaining_steps = TIMESTEPS - len(sequence_X)
        padding = np.zeros((remaining_steps, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS), dtype=np.float32)
        sequence_X = np.concatenate([np.array(sequence_X), padding], axis=0)
        X.append(sequence_X)
    
    X = np.array(X)
    
    # Normalize y values
    y_dict_seq = {}
    for param, values in y_dict.items():
        sequences = [values[i:i+TIMESTEPS] for i in range(0, len(values), TIMESTEPS)]
        if len(sequences[-1]) < TIMESTEPS:
            sequences[-1] = np.pad(sequences[-1], (0, TIMESTEPS - len(sequences[-1])), mode='constant', constant_values=0)
        sequences = np.array(sequences)
        scaler = StandardScaler()
        flat_values = sequences.reshape(-1, 1)
        scaled_values = scaler.fit_transform(flat_values).reshape(sequences.shape)
        y_dict_seq[param] = scaled_values
        scalers[param] = scaler
    
    print(f"Final X shape: {X.shape}")
    return X, y_dict_seq, scalers

X, y_dict, scalers = prepare_dataset(image_data_list)
print(f"Dataset shape: X={X.shape}, y_dict sample shape: {y_dict['AQI'].shape}")

Final X shape: (443, 24, 32, 32, 3)
Dataset shape: X=(443, 24, 32, 32, 3), y_dict sample shape: (443, 24)


In [4]:
def build_cnn_lstm_model(input_shape=(TIMESTEPS, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)):
    """CNN-LSTM for 32x32 whole images."""
    model = Sequential([
        TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=input_shape),
        TimeDistributed(MaxPooling2D((2, 2))),  # 32x32 -> 16x16
        TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
        TimeDistributed(MaxPooling2D((2, 2))),  # 16x16 -> 8x8
        TimeDistributed(Flatten()),  # 8x8x64 = 4096 features per timestep
        LSTM(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=['mae'])
    return model

model = build_cnn_lstm_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 24, 32, 32, 32)   896       
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 24, 16, 16, 32)   0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 24, 16, 16, 64)   18496     
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, 24, 8, 8, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 24, 4096)         0

In [ ]:
def train_models(X, y_dict):
    models = {}
    histories = {}
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    for param, y_values in y_dict.items():
        print(f"\nTraining model for {param}")
        X_train, X_test, y_train, y_test = train_test_split(X, y_values, test_size=0.2, random_state=42)
        print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")
        
        model = build_cnn_lstm_model()
        history = model.fit(
            X_train, y_train[:, -1],
            validation_data=(X_test, y_test[:, -1]),
            epochs=ECHPOS_SIZE,
            batch_size=BATCH_SIZE,
            callbacks=[early_stopping],
            verbose=1
        )
        models[param] = model
        histories[param] = history
        
        model_path = os.path.join("saved_models", f"model_{param}.h5")
        model.save(model_path)
        print(f"Saved model for {param} to {model_path}")
    
    return models, histories

models, histories = train_models(X, y_dict)


Training model for AQI
Training samples: 354, Testing samples: 89
Epoch 1/150
27/36 [=====================>........] - ETA: 12s - loss: 1.0443 - mae: 0.8438

In [ ]:
plt.figure(figsize=(12, 6))
for param, history in histories.items():
    plt.plot(history.history['mae'], label=f'{param} Train MAE')
    plt.plot(history.history['val_mae'], label=f'{param} Val MAE', linestyle='--')
plt.title('Training and Validation MAE Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def test_model(models, X_test=None, y_test_dict=None, scalers=None, new_image_path=None):
    results = {}
    
    if X_test is not None and y_test_dict is not None:
        print(f"\nEvaluating on test data (sequences: {X_test.shape[0]})")
        for param, model in models.items():
            test_loss, test_mae = model.evaluate(X_test, y_test_dict[param][:, -1], verbose=0)
            y_pred_scaled = model.predict(X_test, verbose=0).flatten()
            y_pred = scalers[param].inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
            y_true = scalers[param].inverse_transform(y_test_dict[param][:, -1].reshape(-1, 1)).flatten()
            r2 = r2_score(y_true, y_pred)
            results[param] = {'MSE': test_loss, 'MAE': test_mae, 'R²': r2}
            print(f"{param} - Test MSE (scaled): {test_loss:.4f}, Test MAE (scaled): {test_mae:.4f}, R²: {r2:.4f}")
            print(f"Sample predictions (unscaled): {y_pred[:5]}, Actual: {y_true[:5]}")
    
    elif new_image_path is not None:
        print("\nPredicting for a new sequence:")
        img_resized = load_and_preprocess_image(new_image_path)
        sky_img_seq = np.expand_dims(img_resized, axis=0)
        sky_img_seq = np.repeat(sky_img_seq[np.newaxis, :], TIMESTEPS, axis=1)
        
        for param, model in models.items():
            pred_scaled = model.predict(sky_img_seq, verbose=0)[0]
            pred = scalers[param].inverse_transform([[pred_scaled]])[0][0]
            results[param] = pred
            print(f"Predicted {param}: {pred:.4f}")
    
    return results

X_train, X_test, y_train_dict, y_test_dict = {}, {}, {}, {}
for param, y_values in y_dict.items():
    X_train[param], X_test[param], y_train_dict[param], y_test_dict[param] = train_test_split(
        X, y_values, test_size=0.2, random_state=42
    )

test_results = test_model(models, X_test['AQI'], y_test_dict, scalers)
new_image = "F:/Pheonix/Thesis/Project/Code/AQI Index/.venv/Dev/Data/test_image.jpg"
pred_results = test_model(models, new_image_path=new_image, scalers=scalers)

In [ ]:
img = load_and_preprocess_image(new_image)

plt.figure(figsize=(10, 5))
plt.imshow(img)
plt.title("Original Image")
plt.axis('off')
plt.suptitle("Sample Image with Predictions:\n" + 
             "\n".join(f"{param}: {value:.2f}" for param, value in pred_results.items()))
plt.show()